# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citydf = pd.read_csv('cities.csv', encoding = 'utf-8')
clean_citydf = citydf.replace('',np.nan,inplace=True)
citydf.head()

,City,Country,Date,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed
0,victoria,sc,1.595558e+09,22.29,114.16,91.99,66.0,71.0,8.99
1,vanavara,ru,1.595558e+09,60.34,102.28,58.15,73.0,100.0,9.19
2,borzya,ru,1.595558e+09,50.38,116.52,83.17,49.0,83.0,6.67
3,kavaratti,in,1.595558e+09,10.57,72.64,82.24,80.0,100.0,12.39
4,rikitea,pf,1.595558e+09,-23.12,-134.97,67.24,59.0,39.0,14.07


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Set attributes for the map
city_loc = citydf[['Latitude','Longitude']]
humidity = citydf['Humidity (%)']
hum_max = humidity.max()

# Configure map for gmaps
hum_map = gmaps.figure()
hum_heat = gmaps.heatmap_layer(city_loc, weights = humidity, max_intensity = hum_max)
hum_map.add_layer(hum_heat)
hum_map


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Set the criteria for locations with ideal weather
ideal_weather = citydf.loc[(citydf['Max Temp (F)']>70) & 
                           (citydf['Max Temp (F)']<80) & 
                           (citydf['Wind Speed']<10) & 
                           (citydf['Cloudiness (%)']==0)]
ideal_weather = ideal_weather[['City','Country','Latitude','Longitude']]

# Remove duplicate coordinates (not sure why this happened but I cleaned the df before printing)
ideal_weather.drop_duplicates(['City'], inplace = True)
ideal_weather

,City,Country,Latitude,Longitude
98,ladario,br,-19.00,-57.60
169,porto murtinho,br,-21.70,-57.88
366,luderitz,na,-26.65,15.16
480,kalat,pk,29.02,66.59
557,severinia,br,-20.81,-48.80
612,abu samrah,qa,35.30,37.18
660,chitral,pk,35.85,71.79
669,korla,cn,41.76,86.15
849,awjilah,ly,29.11,21.29
872,tiznit,ma,29.58,-9.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = []

# Create dict for api call
locations = pd.DataFrame({'Latitude': ideal_weather['Latitude'], 'Longitude': ideal_weather['Longitude']})

target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index,row in locations.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    
    # Sets parameters for the api query.
    # After inspecting the query, I noticed that the 'types' were not actually hotels.
    # I researched alternative gmap 'types' in the api documentation.
    # Using 'lodging' returned the best results.
    params = {'location': f'{latitude},{longitude}',
              'radius': 5000,
              'types': 'lodging',
              'key': gkey
             }
    
    response = requests.get(target_url, params = params).json()
    
    try:
        hotel_df.append(response['results'][0]['name'])
    except:
        # used to keep the hotel_df the same length as the source ideal_weather dataframe.
        hotel_df.append("")
        # prints the locations that returned no results
        print(f'No hotel for {latitude, longitude}.')

# Used to validate that the hotel field contained hotels. 
# response

No hotel for (35.3, 37.18).
No hotel for (29.11, 21.29).
No hotel for (29.58, -9.5).


['Hotel Fazenda 4 cantos',
 'Hotel Pantanal',
 'LÜDERITZ NEST HOTEL',
 'Balochistan Hotel',
 'Oasis Hotel',
 '',
 'PTDC Motel Chitral',
 'Gulisitan Hotel',
 '',
 '',
 'Hotel Roscianum Wellness & SPA',
 'Hotel Mirage',
 'Hotel Posta',
 'Gostinitsa Alrosa',
 'Grand Ramela Hotel',
 'Casa Rural Entre Viñas']

In [8]:
# Add the hotel list to the main df
ideal_weather['Hotel Name'] = hotel_df

# Clean the rows with no hotels
ideal_weather.replace('',np.nan,inplace=True)
ideal_weather_cleaned = ideal_weather.dropna()

# Create a dataframe with only 10 rows
top_10_hotels = ideal_weather_cleaned.head(10)
top_10_hotels


,City,Country,Latitude,Longitude,Hotel Name
98,ladario,br,-19.00,-57.60,Hotel Fazenda 4 cantos
169,porto murtinho,br,-21.70,-57.88,Hotel Pantanal
366,luderitz,na,-26.65,15.16,LÜDERITZ NEST HOTEL
480,kalat,pk,29.02,66.59,Balochistan Hotel
557,severinia,br,-20.81,-48.80,Oasis Hotel
660,chitral,pk,35.85,71.79,PTDC Motel Chitral
669,korla,cn,41.76,86.15,Gulisitan Hotel
879,rossano,it,39.58,16.63,Hotel Roscianum Wellness & SPA
1106,vilhena,br,-12.74,-60.15,Hotel Mirage
1161,syracuse,it,37.09,15.28,Hotel Posta


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in top_10_hotels.iterrows()]
locations = top_10_hotels[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
hotels = gmaps.marker_layer(locations)
hum_map.add_layer(hotels)
hum_map

# Display figure


Figure(layout=FigureLayout(height='420px'))